# Introdução

## Importação das bibliotecas necessarias

In [7]:
%matplotlib inline
import requests
import pandas as pd
import os
import json
import sys
import matplotlib.pyplot as plt
import seaborn as sns
import random as random
import numpy as np
import ipywidgets as widgets
import glob, os
sns.set()

## Importação das tabelas

In [76]:
df_2015 = pd.read_csv('../csv_files/rmm/nascidos_vivos_2015.csv', sep=';')
df_2016 = pd.read_csv('../csv_files/rmm/nascidos_vivos_2016.csv', sep=';')
df_2017 = pd.read_csv('../csv_files/rmm/nascidos_vivos_2017.csv', sep=';')
df_2018 = pd.read_csv('../csv_files/rmm/nascidos_vivos_2018.csv', sep=';')
df_2019 = pd.read_csv('../csv_files/rmm/nascidos_vivos_2019.csv', sep=';')
df_rmm = pd.read_csv('../csv_files/QW-20-razao_mortalidade_materna.csv')
df_rmm_direta = pd.read_csv('../csv_files/QW-21-razao_mortalidade_materna_direta.csv')

## Ajuste dos anos de 2015 a 2019 a partir Estimativa do IBGE

In [77]:
df_rmm.loc[(df_rmm['CODMUNRES'].isin([330455]))]

,CODMUNRES,Ano,Qtd_Morte_Materna,Qtd_Total,Ratio
26031,330455,2012,59.0,86549.0,68.169476
26032,330455,2013,67.0,87661.0,76.430796
26033,330455,2014,64.0,89991.0,71.118223
26034,330455,2015,65.0,90694.0,71.669570
26035,330455,2016,62.0,83166.0,74.549696
26036,330455,2017,70.0,84521.0,82.819654
26037,330455,2018,50.0,82554.0,60.566417
26038,330455,2019,62.0,76803.0,80.726013


In [78]:
df_rmm_direta.loc[(df_rmm['CODMUNRES'].isin([330455]))]

,CODMUNRES,Ano,Qtd_Morte_Materna_Direta,Qtd_Total,Ratio
26031,330455,2012,36.0,86549,41.594935
26032,330455,2013,35.0,87661,39.926535
26033,330455,2014,32.0,89991,35.559111
26034,330455,2015,45.0,90694,49.617395
26035,330455,2016,32.0,83166,38.477262
26036,330455,2017,40.0,84521,47.325517
26037,330455,2018,26.0,82554,31.494537
26038,330455,2019,34.0,76803,44.269104


In [83]:
#df_rmm
df_2015['Código Município'] = df_2015['Código Município'].astype(str)
df_2016['Código Município'] = df_2016['Código Município'].astype(str)
df_2017['Código Município'] = df_2017['Código Município'].astype(str)
df_2018['Código Município'] = df_2018['Código Município'].astype(str)
df_2019['Código Município'] = df_2019['Código Município'].astype(str)

df_rmm['CODMUNRES'] = df_rmm['CODMUNRES'].astype(str)
df_rmm_direta['CODMUNRES'] = df_rmm_direta['CODMUNRES'].astype(str)

for i in range(2015, 2020):
    for index, row in globals()['df_'+str(i)].iterrows():
        if(len(row[0]) == 7):
            mun = row[0][:-1]
        else:
            mun = row[0]
        idx = df_rmm.loc[(df_rmm['CODMUNRES'].isin([mun])) & (df_rmm['Ano'] == i)].index[0]
        df_rmm.at[idx, 'Qtd_Total']=row[1]
        idx = df_rmm_direta.loc[(df_rmm_direta['CODMUNRES'].isin([mun])) & (df_rmm_direta['Ano'] == i)].index[0]
        df_rmm_direta.at[idx, 'Qtd_Total']=row[1]
    print('foi '+str(i))
print('foi tudo')


foi 2015
foi 2016
foi 2017
foi 2018
foi 2019
foi tudo


In [88]:
df_rmm['Ratio'] = ((df_rmm['Qtd_Morte_Materna']/df_rmm['Qtd_Total'])*100000)
df_rmm.loc[(df_rmm['CODMUNRES'].isin(['330455']))]

,CODMUNRES,Ano,Qtd_Morte_Materna,Qtd_Total,Ratio
26031,330455,2012,59.0,86549.0,68.169476
26032,330455,2013,67.0,87661.0,76.430796
26033,330455,2014,64.0,89991.0,71.118223
26034,330455,2015,65.0,96059.0,67.666746
26035,330455,2016,62.0,84638.0,73.253149
26036,330455,2017,70.0,85132.0,82.225250
26037,330455,2018,50.0,83138.0,60.140970
26038,330455,2019,62.0,77137.0,80.376473


In [89]:
df_rmm_direta['Ratio'] = ((df_rmm_direta['Qtd_Morte_Materna_Direta']/df_rmm_direta['Qtd_Total'])*100000)
df_rmm_direta.loc[(df_rmm_direta['CODMUNRES'].isin(['330455']))]

,CODMUNRES,Ano,Qtd_Morte_Materna_Direta,Qtd_Total,Ratio
26031,330455,2012,36.0,86549,41.594935
26032,330455,2013,35.0,87661,39.926535
26033,330455,2014,32.0,89991,35.559111
26034,330455,2015,45.0,96059,46.846209
26035,330455,2016,32.0,84638,37.808077
26036,330455,2017,40.0,85132,46.985857
26037,330455,2018,26.0,83138,31.273305
26038,330455,2019,34.0,77137,44.077421


In [91]:
df_rmm.to_csv('../csv_files/QW-20C-razao_mortalidade_materna.csv',index=False)
df_rmm_direta.to_csv('../csv_files/QW-21C-razao_mortalidade_materna_direta.csv',index=False)

## Sorteamento dos municípios pelas faixas